<a href="https://colab.research.google.com/github/Isabellitankian/FIAP---2023/blob/main/RN_CNNTransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 17 - CNN + Transfer Learning

In [ ]:
# Imports
import tensorflow as tf
import numpy as np

In [ ]:
# Carregar o dataset
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"],
    batch_size=-1, 
    as_supervised=True
)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to ~/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [ ]:
train_ds.shape

TensorShape([2569, 442, 1024, 3])

In [ ]:
test_ds.shape

TensorShape([1101, 442, 1024, 3])

## Processar os dados

In [ ]:
size = (150 , 150)

train_ds = tf.image.resize(train_ds, size)
test_ds = tf.image.resize(test_ds, size)

train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [ ]:
train_labels

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

## Modelo "profundo" simples

In [ ]:
# Import 
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

# Definir a arquitetura da CNN -> camada de extração de features
hand_model = Sequential()
hand_model.add( Rescaling(1./255, input_shape=(150, 150, 3)) )

hand_model.add( layers.Conv2D(16, kernel_size=10, activation="relu") )
hand_model.add( layers.MaxPooling2D(3) ) 

hand_model.add( layers.Conv2D(32, kernel_size=8, activation="relu") )
hand_model.add( layers.MaxPooling2D(2) ) 

hand_model.add( layers.Conv2D(32, kernel_size=6, activation="relu") )
hand_model.add( layers.MaxPooling2D(2) ) 

# Definir a arquitetura da CNN -> camada de decisão (classificação)
hand_model.add( layers.Flatten() )
hand_model.add( layers.Dense(50, activation="relu") )
hand_model.add( layers.Dense(20, activation="relu") )
hand_model.add( layers.Dense(5, activation="softmax") )

# Compila o modelo
hand_model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Definir a estratégia de Early Stopping
es = EarlyStopping(monitor="val_accuracy", 
                   patience=5, 
                   restore_best_weights=True)

# Treinamento do modelo 
with tf.device("/device:GPU:0"):
  hand_model.fit(train_ds, train_labels,
                 epochs=50,
                 validation_split=0.2,
                 batch_size=32,
                 callbacks=[es])

Epoch 1/50
65/65 [==============================] - 16s 33ms/step - loss: 1.5998 - accuracy: 0.2516 - val_loss: 1.6026 - val_accuracy: 0.2860
Epoch 2/50
65/65 [==============================] - 2s 32ms/step - loss: 1.4961 - accuracy: 0.3309 - val_loss: 1.3953 - val_accuracy: 0.3911
Epoch 3/50
65/65 [==============================] - 2s 24ms/step - loss: 1.3104 - accuracy: 0.4307 - val_loss: 1.3321 - val_accuracy: 0.4202
Epoch 4/50
65/65 [==============================] - 2s 24ms/step - loss: 1.2770 - accuracy: 0.4540 - val_loss: 1.2812 - val_accuracy: 0.4436
Epoch 5/50
65/65 [==============================] - 2s 25ms/step - loss: 1.1836 - accuracy: 0.4983 - val_loss: 1.2218 - val_accuracy: 0.4669
Epoch 6/50
65/65 [==============================] - 2s 27ms/step - loss: 1.1556 - accuracy: 0.5119 - val_loss: 1.2270 - val_accuracy: 0.5058
Epoch 7/50
65/65 [==============================] - 3s 50ms/step - loss: 1.0809 - accuracy: 0.5513 - val_loss: 1.1443 - val_accuracy: 0.5195
Epoch 8/50
6

In [ ]:
# Avaliação do modelo
loss, acc = hand_model.evaluate(test_ds, test_labels)

print(f"A loss do modelo é {loss:.2f} e a ACC é: {acc:.2f}")

35/35 [==============================] - 1s 18ms/step - loss: 0.7308 - accuracy: 0.7130
A loss do modelo é 0.73 e a ACC é: 0.71


In [ ]:
# Michel TODO - trazer a matriz de confusão e gráficos de loss/accuracy